In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [18]:
import requests
from langchain_core.tools import tool

In [28]:
@tool
def get_conversion_factor(base_currency: str, target_currency: str)-> str:
    """This function fetches the currency conversion factor from given base currency to target currency"""
    url = f"https://v6.exchangerate-api.com/v6/a411f1bec9afb8ef0ef05dd6/pair/{base_currency}/{target_currency}"

    response = requests.get(url)
    return response.json()

In [29]:
@tool
def convert(base_currency_value: int, conversion_rate: float)-> float:
    """Given a currency conversion rate this function calculates the target currency value from a base currency value"""
    return base_currency_value * conversion_rate

In [30]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1747267202,
 'time_last_update_utc': 'Thu, 15 May 2025 00:00:02 +0000',
 'time_next_update_unix': 1747353602,
 'time_next_update_utc': 'Fri, 16 May 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4468}

In [31]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999